In [54]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms, models
import torch.nn.functional as F
from torchvision.models import resnet18
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import os
import copy



device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

model = resnet18(pretrained=True)
model = model.to(device)

cuda


In [41]:

preprocess = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [3]:
#download the dataset cifar10, and create a dataloader with appropriate preprocessing
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=100, shuffle=True, num_workers=1)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=1)

0.0%

100.0%


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [48]:
processed_trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=preprocess)
class_names = processed_trainset.classes
processed_testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=preprocess)
dataloaders = {'train' : [], 'val' : []}
dataloaders['train'] = torch.utils.data.DataLoader(processed_trainset, batch_size=100,
                                                    shuffle=True, num_workers=4, pin_memory=True)
dataloaders['val'] = torch.utils.data.DataLoader(processed_testset, batch_size=100,
                                                    shuffle=False, num_workers=4, pin_memory=True)
dataset_sizes = {'train' : len(processed_trainset), 'val' : len(processed_testset)}

Files already downloaded and verified
Files already downloaded and verified


In [15]:
#Finetune to Cifar10: Load the resnet18 model that has been pretrained on Imagenet.
# Set requires_grad=False for all of the weights. Change its last layer (“fc”) to have only 10 outputs. The last layer weights will have requires_grad=True. 
for param in model.parameters():
    param.requires_grad = False
model.fc = nn.Linear(512, 10)

In [33]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
dataset_sizes = 50,000

In [52]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=50):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [45]:
model_conv = train_model(model, criterion, optimizer,
                         exp_lr_scheduler, num_epochs=5)

Epoch 0/4
----------
train Loss: 1.5574 Acc: 0.4606
val Loss: 1.3334 Acc: 0.5427

Epoch 1/4
----------
train Loss: 1.2765 Acc: 0.5629
val Loss: 1.2346 Acc: 0.5718

Epoch 2/4
----------
train Loss: 1.2151 Acc: 0.5782
val Loss: 1.1863 Acc: 0.5880

Epoch 3/4
----------
train Loss: 1.1860 Acc: 0.5898
val Loss: 1.1811 Acc: 0.5905

Epoch 4/4
----------
train Loss: 1.1875 Acc: 0.5917
val Loss: 1.1880 Acc: 0.5854

Training complete in 8m 6s
Best val Acc: 0.590500


In [53]:
#Use the cifar 10 training dataset to train the modified model for say 50 epochs.
model_conv = train_model(model, criterion, optimizer,
                         exp_lr_scheduler, num_epochs=50)

Epoch 0/49
----------
train Loss: 1.2134 Acc: 0.5857
val Loss: 1.1957 Acc: 0.5881

Epoch 1/49
----------
train Loss: 1.2064 Acc: 0.5843
val Loss: 1.2004 Acc: 0.5920

Epoch 2/49
----------
train Loss: 1.1974 Acc: 0.5914
val Loss: 1.1925 Acc: 0.5825

Epoch 3/49
----------
train Loss: 1.1894 Acc: 0.5917
val Loss: 1.1723 Acc: 0.5935

Epoch 4/49
----------
train Loss: 1.1913 Acc: 0.5903
val Loss: 1.1740 Acc: 0.5916

Epoch 5/49
----------
train Loss: 1.1903 Acc: 0.5911
val Loss: 1.1885 Acc: 0.5886

Epoch 6/49
----------
train Loss: 1.1914 Acc: 0.5931
val Loss: 1.1783 Acc: 0.5935

Epoch 7/49
----------
train Loss: 1.1905 Acc: 0.5906
val Loss: 1.1818 Acc: 0.5912

Epoch 8/49
----------
train Loss: 1.1902 Acc: 0.5891
val Loss: 1.1711 Acc: 0.5971

Epoch 9/49
----------
train Loss: 1.1892 Acc: 0.5889
val Loss: 1.1856 Acc: 0.5945

Epoch 10/49
----------
train Loss: 1.1868 Acc: 0.5902
val Loss: 1.1702 Acc: 0.5980

Epoch 11/49
----------
train Loss: 1.1873 Acc: 0.5923
val Loss: 1.1890 Acc: 0.5913

Ep

In [55]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    model_ft = models.resnet18(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, num_classes)
    input_size = 224

    return model_ft, input_size

In [59]:
scratch_model,_ = initialize_model(model, num_classes = 10, feature_extract=False, use_pretrained=False)
scratch_model = scratch_model.to(device)
scratch_optimizer = optim.SGD(scratch_model.parameters(), lr=0.001, momentum=0.9)
scratch_criterion = nn.CrossEntropyLoss()
_,scratch_hist = train_model(model, criterion, optimizer,
                         exp_lr_scheduler, num_epochs=15)

Epoch 0/14
----------
train Loss: 9.7918 Acc: 0.0013
val Loss: 9.8673 Acc: 0.0016

Epoch 1/14
----------
train Loss: 9.7786 Acc: 0.0013
val Loss: 9.8887 Acc: 0.0011

Epoch 2/14
----------
train Loss: 9.7768 Acc: 0.0014
val Loss: 9.7829 Acc: 0.0011

Epoch 3/14
----------
train Loss: 9.7830 Acc: 0.0012
val Loss: 9.8845 Acc: 0.0015

Epoch 4/14
----------
train Loss: 9.7909 Acc: 0.0012
val Loss: 9.8258 Acc: 0.0011

Epoch 5/14
----------
train Loss: 9.7845 Acc: 0.0012
val Loss: 9.8410 Acc: 0.0019

Epoch 6/14
----------
train Loss: 9.7925 Acc: 0.0015
val Loss: 9.8342 Acc: 0.0015

Epoch 7/14
----------
train Loss: 9.7825 Acc: 0.0011
val Loss: 9.8381 Acc: 0.0018

Epoch 8/14
----------
train Loss: 9.7891 Acc: 0.0012
val Loss: 9.7886 Acc: 0.0015

Epoch 9/14
----------
train Loss: 9.7762 Acc: 0.0014
val Loss: 9.7538 Acc: 0.0016

Epoch 10/14
----------
train Loss: 9.7815 Acc: 0.0011
val Loss: 9.8740 Acc: 0.0015

Epoch 11/14
----------
train Loss: 9.7915 Acc: 0.0013
val Loss: 9.8090 Acc: 0.0018

Ep

TypeError: cannot unpack non-iterable ResNet object